In [10]:
import gc
import pandas as pd
from sqlalchemy import create_engine, URL
from tqdm import tqdm

In [11]:
df_patstat = pd.read_json('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/df_patstat_cleantech_granted_abstract_metadata.json')
df_rel_on_science = pd.read_json('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_cleantech_lang_detect_yake_title_abstract_noun_chunks.json')
df_rel_pcs = pd.read_csv("/mnt/hdd01/Reliance on Science/Raw Files/_pcs_oa.csv")

In [12]:
df_rel_pcs['patent_id_prefix'] = df_rel_pcs['patent'].apply(lambda x: x.split('-')[0])
df_rel_pcs['patent_id'] = df_rel_pcs['patent'].apply(lambda x: '-'.join(x.split('-')[1:-1]))
df_rel_pcs = df_rel_pcs[df_rel_pcs['patent_id_prefix'].isin(['us', 'ep'])]
df_rel_pcs = df_rel_pcs[df_rel_pcs['oaid'].isin(df_rel_on_science['oaid'])]

In [13]:
patstat_url_object = URL.create(
    drivername="postgresql+psycopg2",
    username="tie",
    password="TIE%2023!tuhh",
    host="134.28.58.100",
    port="65432",
    database="PATSTAT_2023",
)
engine = create_engine(patstat_url_object)

In [14]:
df_rel_pcs['patent_id_prefix'] = df_rel_pcs['patent_id_prefix'].apply(lambda x: x.upper())
df_rel_pcs.to_sql('temp_df_rel_pcs', engine, if_exists='replace', index=False)

607

In [15]:
query = """
SELECT temp.patent_id, temp.patent_id_prefix, publn.appln_id
FROM temp_df_rel_pcs AS temp
INNER JOIN tls211_pat_publn AS publn
ON temp.patent_id = publn.publn_nr AND temp.patent_id_prefix = publn.publn_auth
"""
df_result = pd.read_sql_query(query, engine)

In [16]:
df_result['patent_id_prefix_patent_id'] = df_result['patent_id_prefix'] + '-' + df_result['patent_id']
df_rel_pcs['patent_id_prefix_patent_id'] = df_rel_pcs['patent_id_prefix'] + '-' + df_rel_pcs['patent_id']
df_rel_pcs.drop(columns=['patent', 'wherefound', 'confscore', 'self', 'reftype'], inplace=True)

In [17]:
len(df_rel_pcs), len(df_result)

(13363607, 15258239)

In [18]:
# df_rel_pcs.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_rel_pcs.csv', index=False)
# df_result.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_patstat.csv', index=False)

In [21]:
df_result = df_result[['patent_id_prefix_patent_id', 'appln_id']]
df_rel_pcs = df_rel_pcs[['oaid', 'patent_id_prefix_patent_id']]

df_result.set_index('patent_id_prefix_patent_id', inplace=True)
df_rel_pcs.set_index('patent_id_prefix_patent_id', inplace=True)

df_rel_pcs = df_rel_pcs.join(df_result, how='left')

df_rel_pcs.reset_index(inplace=True)

In [24]:
df_rel_pcs.drop_duplicates(subset=['oaid', 'appln_id'], inplace=True)

In [32]:
df_rel_pcs.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_rel_pcs_patstat.csv', index=False)